In [1]:
import os
import sys
import glob
import yaml

import numpy as np
import pandas as pd

from math import ceil

from matplotlib import cm

import tifffile

import zarr
import napari
import dask.array as da

# add parent directory to sys.path to call utility functions
sys.path.append(os.path.abspath(os.path.join('..')))
from utils.utility_functions import single_channel_pyramid

In [2]:
# connect to "smb://research.files.med.harvard.edu/SysBio"
# to read image data

In [3]:
# specify which clustering(s) in main.csv to visualize
clusterings = ['cluster']

In [4]:
# I/O

# read single-cell data
main = pd.read_csv('/Users/greg/Desktop/MULTI-TISSUE_FINAL/main.csv')

main = main[main['Sample'] == 'CRC102']
tif_path = '/Volumes/SysBio/SORGER PROJECTS/people/Greg_Baker/data_097_102/tif/WD-76845-102.ome.tif'
seg_path = '/Volumes/SysBio/SORGER PROJECTS/people/Greg_Baker/data_097_102/seg/CRC102/nucleiRingOutlines.tif'

# import markers.csv
markers = pd.read_csv('/Users/greg/Desktop/CRC097_102/CRC-097_mcmicro_markers.csv')

# import image contrast settings
# with open(os.path.join(os.getcwd(), 'input/CRC-097_cylinter_contrast_limits.yml')) as f:
#     contrast_limits = yaml.safe_load(f)

# isolate antibodies of interest
abx_channels = [
    "anti_CD3", "anti_CD45RO", "Keratin_570", "aSMA_660", "CD4_488", "CD45_PE", "PD1_647",
    "CD20_488", "CD68_555", "CD8a_660", "CD163_488", "FOXP3_570", "PDL1_647", "Ecad_488",
    "Vimentin_555", "CDX2_647", "LaminABC_488", "Desmin_555", "CD31_647", "PCNA_488",
    "CollagenIV_647"
]

In [5]:
# add DNA1 channel to image viewer
dna, min, max = single_channel_pyramid(glob.glob(tif_path)[0], channel=0)
viewer = napari.view_image(
    dna, rgb=False, blending='additive',
    colormap='gray', visible=True, opacity=0.8,
    name='DNA1', contrast_limits=(min, max)
)

In [6]:
# add marker channels to image viewer and apply previously defined contrast limits
for ch in abx_channels:
    channel_number = markers['channel_number'][markers['marker_name'] == ch]
    
    color = 'green'
    
    img, min, max = single_channel_pyramid(
        glob.glob(tif_path)[0], channel=(channel_number.item() - 1)
    )
    viewer.add_image(
        img, rgb=False, blending='additive', colormap=color, visible=False, name=ch,
        contrast_limits=(min, max)
    )
# for ch in abx_channels:
#     viewer.layers[ch].contrast_limits = (
#         contrast_limits[ch][0], contrast_limits[ch][1])

In [7]:
# add centroids of cells in each clustering
for clustering in clusterings:
    
    num_colors = len(list(cm.tab20.colors))
    num_clusters = len(main[clustering].unique())
    palette_multiplier = ceil(num_clusters / num_colors)
    colors = list(cm.tab20.colors) * palette_multiplier
    colors = colors[0:num_clusters]
    colors.reverse()

    for c, cluster in zip(colors, sorted(main[clustering].unique(), reverse=True)):
        # if cluster in [0, 11, 228, 232]:
        centroids = main[['Y_centroid', 'X_centroid']][main[clustering] == cluster]
        viewer.add_points(
            centroids, name=f'{clustering}_{cluster}', face_color=np.array(c), edge_color='white',
            edge_width=0.0, size=60.0, opacity=1.0, blending='translucent', visible=False
        )

In [8]:
# add segmentation outlines to image viewer
# seg, min, max = single_channel_pyramid(glob.glob(seg_path)[0], channel=0)
# viewer.add_image(
#     seg, rgb=False, blending='additive',
#     colormap='gray', visible=False,
#     name='segmentation', opacity=0.3, contrast_limits=(min, max)
# )

In [9]:
# run image viewer
viewer.scale_bar.visible = True
viewer.scale_bar.unit = 'um'

napari.run()